<a href="https://colab.research.google.com/github/shimonsant2/FinAppV3/blob/main/FinAppV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install yfinance pandas matplotlib requests plotly ipywidgets --quiet

import os
import json
import textwrap
import datetime as dt

import requests
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go

from IPython.display import display

%matplotlib inline

# ---- SEC headers ----
SEC_HEADERS = {
    "User-Agent": "Your Name your.email@example.com",
    "Accept-Encoding": "gzip, deflate"
}

# ---- STEP 1: Find ticker from company name (via SEC) ----
def find_symbol_from_company_name(company_name: str) -> str | None:
    """
    Find symbol by company name using SEC official company list.
    """
    url = "https://www.sec.gov/files/company_tickers.json"
    resp = requests.get(url, headers=SEC_HEADERS)
    resp.raise_for_status()
    data = resp.json()

    company_upper = company_name.upper()
    best_match = None

    for entry in data.values():
        name = entry["title"].upper()
        if company_upper in name:
            best_match = entry["ticker"]
            break

    return best_match

# ---- INPUT COMPANY ----
company_name = "servicenow"
ticker_symbol = find_symbol_from_company_name(company_name)
print("Found symbol:", ticker_symbol)


# ---- STEP 2: Download history ----
def download_price_history(symbol: str, period: str = "10y"):
    """
    Download up to 10 years of historical data for a symbol using yfinance.
    Returns a DataFrame or None if it fails.
    """
    try:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period=period, auto_adjust=False)
    except Exception as e:
        print("Error downloading history:", e)
        return None

    if hist is None or hist.empty:
        print("No history returned for", symbol)
        return None

    hist.index = pd.to_datetime(hist.index)
    return hist


# ---- STEP 2b: Plot with zoom + OHLCV hover ----
def plot_time_ranges_interactive(hist: pd.DataFrame, symbol: str):
    """
    Plot interactive Plotly charts for:
      - 1 Day, 1 Week, 1 Month, 1 Year, 2 Years, 3 Years, 5 Years, 10 Years.

    For each chart:
      - Title includes % change over the full range (Δ +X.XX%).
      - Hover tooltip shows Open, High, Low, Close, Volume.
      - Zoom / pan via toolbar + range slider.
      - 'Reset View' button resets axes to the original full range.

    NOTE: In Colab we CANNOT update the title automatically based on mouse selection.
    Use the helper function below to compute % change for any chosen period.
    """
    hist = hist.copy()
    end = hist.index.max()

    def slice_range(days):
        if days is None:
            return hist
        start = end - pd.Timedelta(days=days)
        return hist.loc[hist.index >= start]

    ranges = {
        "1 Day": 1,
        "1 Week": 7,
        "1 Month": 30,
        "1 Year": 365,
        "2 Years": 365 * 2,
        "3 Years": 365 * 3,
        "5 Years": 365 * 5,
        "10 Years": None,  # full history
    }

    for label, days in ranges.items():
        sliced = slice_range(days)
        if sliced.empty:
            print(f"{label}: Not enough data to plot.")
            continue

        # Ensure OHLCV exists
        required_cols = ["Open", "High", "Low", "Close", "Volume"]
        if not all(col in sliced.columns for col in required_cols):
            print(f"{label}: Missing OHLCV columns in data.")
            continue

        close_series = sliced["Close"].dropna()
        if close_series.empty:
            print(f"{label}: No valid close prices.")
            continue

        # Full-range % change (Close)
        start_price = close_series.iloc[0]
        end_price = close_series.iloc[-1]
        if pd.notna(start_price) and pd.notna(end_price) and start_price != 0:
            pct_change_full = (end_price / start_price - 1) * 100
            pct_text_full = f"{pct_change_full:+.2f}%"
        else:
            pct_text_full = "N/A"

        base_title = f"{symbol} – Close Price – {label} (Δ {pct_text_full})"

        # Add OHLCV to hover via customdata
        customdata = sliced[["Open", "High", "Low", "Close", "Volume"]].values

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=sliced.index,
                y=sliced["Close"],
                mode="lines",
                name=symbol,
                customdata=customdata,
                hovertemplate=(
                    "Date: %{x}<br>"
                    "Open: %{customdata[0]:.2f}<br>"
                    "High: %{customdata[1]:.2f}<br>"
                    "Low: %{customdata[2]:.2f}<br>"
                    "Close: %{customdata[3]:.2f}<br>"
                    "Volume: %{customdata[4]:,.0f}<extra></extra>"
                ),
            )
        )

        fig.update_layout(
            title=base_title,
            xaxis_title="Date",
            yaxis_title="Price",
            hovermode="x unified",
            height=400,
            xaxis=dict(
                rangeslider=dict(visible=True),
                type="date"
            ),
            updatemenus=[
                dict(
                    type="buttons",
                    direction="left",
                    buttons=[
                        dict(
                            label="Reset View",
                            method="relayout",
                            args=[{"xaxis.autorange": True, "yaxis.autorange": True}],
                        )
                    ],
                    x=1,
                    xanchor="right",
                    y=1.15,
                    yanchor="top",
                    showactive=False,
                )
            ],
        )

        print(f"\n=== {label} ===")
        print("Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.")
        print("Click 'Reset View' on the chart to return to full range.")
        fig.show()


# ---- STEP 2c: Helper to compute % for a specific period ----
def percent_change_between_dates(hist: pd.DataFrame, symbol: str, start_date: str, end_date: str):
    """
    Compute percentage change in Close price between two dates and print
    a 'virtual header' that describes the selected period.

    Parameters:
      hist       : full history DataFrame (e.g. hist_10y)
      symbol     : ticker symbol
      start_date : string like '2022-01-01'
      end_date   : string like '2023-03-15'
    """
    if hist is None or hist.empty:
        print("History is empty.")
        return

    hist_sorted = hist.sort_index()

    # Convert date strings to timestamps
    start_ts = pd.to_datetime(start_date)
    end_ts = pd.to_datetime(end_date)

    try:
        # Get nearest available indices for the requested dates
        start_idx = hist_sorted.index.get_loc(start_ts, method="nearest")
        end_idx = hist_sorted.index.get_loc(end_ts, method="nearest")
    except Exception as e:
        print("Error locating dates:", e)
        return

    # Ensure start_idx <= end_idx
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx

    start_row = hist_sorted.iloc[start_idx]
    end_row = hist_sorted.iloc[end_idx]

    start_price = start_row["Close"]
    end_price = end_row["Close"]

    if pd.isna(start_price) or pd.isna(end_price) or start_price == 0:
        print("Cannot compute percentage change due to missing/invalid prices.")
        return

    pct = (end_price / start_price - 1) * 100
    abs_change = end_price - start_price

    start_actual = hist_sorted.index[start_idx].date()
    end_actual = hist_sorted.index[end_idx].date()

    # This line mimics the "title" you wanted
    header = (
        f"{symbol} – Selected Period | {start_actual} → {end_actual} "
        f"(Δ {abs_change:+.2f}, {pct:+.2f}%)"
    )
    print(header)


# ---- RUN: download history + plot ----
if ticker_symbol is None:
    print("Ticker symbol not found. Check company_name.")
else:
    hist_10y = download_price_history(ticker_symbol, period="10y")
    if hist_10y is not None:
        plot_time_ranges_interactive(hist_10y, ticker_symbol)
    else:
        print("Could not download history for ticker:", ticker_symbol)


# ==== STEP 3: SEC filings fetch & download (2 x 10-K, 4 x 10-Q) ==== #

def get_cik_for_ticker(ticker: str):
    """
    Get zero-padded 10-digit CIK using SEC company_tickers.json.
    Returns cik string or None.
    """
    url = "https://www.sec.gov/files/company_tickers.json"
    try:
        resp = requests.get(url, headers=SEC_HEADERS, timeout=30)
        resp.raise_for_status()
    except Exception as e:
        print("Error fetching SEC ticker list:", e)
        return None

    data = resp.json()
    t_upper = ticker.upper()

    for entry in data.values():
        if entry.get("ticker", "").upper() == t_upper:
            cik_int = int(entry["cik_str"])
            return f"{cik_int:010d}"

    print("CIK not found for ticker:", ticker)
    return None


def get_company_submissions(cik: str):
    """
    Get SEC submissions JSON for a company CIK.
    """
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    try:
        resp = requests.get(url, headers=SEC_HEADERS, timeout=30)
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        print("Error fetching company submissions:", e)
        return None


def pick_filings(submissions: dict, form_type: str, limit: int):
    """
    Select up to 'limit' most recent filings of a certain form (e.g. 10-K, 10-Q).
    Returns a list of dicts with basic metadata.
    """
    if submissions is None:
        return []

    recent = submissions.get("filings", {}).get("recent", {})
    forms = recent.get("form", [])
    accessions = recent.get("accessionNumber", [])
    dates = recent.get("filingDate", [])
    docs = recent.get("primaryDocument", [])

    rows = []
    for f, acc, d, doc in zip(forms, accessions, dates, docs):
        if f == form_type:
            rows.append({
                "form": f,
                "accession": acc,
                "date": d,
                "doc": doc,
            })

    return rows[:limit]


def build_filing_url(cik: str, accession: str, primary_doc: str):
    """
    Build the full EDGAR URL for a given filing.
    """
    cik_nolead = cik.lstrip("0")
    acc_nodash = accession.replace("-", "")
    return f"https://www.sec.gov/Archives/edgar/data/{cik_nolead}/{acc_nodash}/{primary_doc}"


def download_filings(cik: str, filings: list, dest_dir: str):
    """
    Download filings into dest_dir.
    Returns list of metadata with 'path' and 'url'.
    """
    os.makedirs(dest_dir, exist_ok=True)
    downloaded = []

    for f in filings:
        url = build_filing_url(cik, f["accession"], f["doc"])
        filename = f"{f['form']}_{f['accession'].replace('-', '')}_{f['doc'].replace('/', '_')}"
        path = os.path.join(dest_dir, filename)

        print("Downloading:", url)
        try:
            r = requests.get(url, headers=SEC_HEADERS, timeout=60)
            r.raise_for_status()
            with open(path, "wb") as fp:
                fp.write(r.content)

            downloaded.append({
                "form": f["form"],
                "date": f["date"],
                "accession": f["accession"],
                "url": url,
                "path": path,
            })
        except Exception as e:
            print("Error downloading filing:", url, "-", e)

    return downloaded


# ---- Run Step 3 for current ticker ----
if ticker_symbol is None:
    print("No ticker symbol – cannot fetch SEC filings.")
else:
    cik = get_cik_for_ticker(ticker_symbol)
    print("Ticker:", ticker_symbol, "| CIK:", cik)

    if cik is not None:
        submissions = get_company_submissions(cik)

        # A. 2 latest annual (10-K)
        latest_10k = pick_filings(submissions, "10-K", limit=2)

        # B. 4 latest quarterly (10-Q)
        latest_10q = pick_filings(submissions, "10-Q", limit=4)

        print("\nLatest 10-K metadata:")
        display(pd.DataFrame(latest_10k))

        print("\nLatest 10-Q metadata:")
        display(pd.DataFrame(latest_10q))

        downloaded_10k = download_filings(cik, latest_10k, dest_dir="sec_10k")
        downloaded_10q = download_filings(cik, latest_10q, dest_dir="sec_10q")

        print("\nSaved 10-K files:")
        for d in downloaded_10k:
            print(d["date"], d["form"], "→", d["path"])

        print("\nSaved 10-Q files:")
        for d in downloaded_10q:
            print(d["date"], d["form"], "→", d["path"])
    else:
        latest_10k = []
        latest_10q = []
        print("CIK not found – skipping SEC download.")



Found symbol: NOW

=== 1 Day ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 1 Week ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 1 Month ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 1 Year ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 2 Years ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.


...

Ticker: NOW | CIK: 0001373715

Latest 10-K metadata:


,form,accession,date,doc
0,10-K,0001373715-25-000010,2025-01-30,now-20241231.htm
1,10-K,0001373715-24-000030,2024-01-25,now-20231231.htm



Latest 10-Q metadata:


,form,accession,date,doc
0,10-Q,0001373715-25-000309,2025-10-30,now-20250930.htm
1,10-Q,0001373715-25-000276,2025-07-24,now-20250630.htm
2,10-Q,0001373715-25-000126,2025-04-23,now-20250331.htm
3,10-Q,0001373715-24-000344,2024-10-24,now-20240930.htm


Downloading: https://www.sec.gov/Archives/edgar/data/1373715/000137371525000010/now-20241231.htm
Downloading: https://www.sec.gov/Archives/edgar/data/1373715/000137371524000030/now-20231231.htm
Downloading: https://www.sec.gov/Archives/edgar/data/1373715/000137371525000309/now-20250930.htm
Downloading: https://www.sec.gov/Archives/edgar/data/1373715/000137371525000276/now-20250630.htm
Downloading: https://www.sec.gov/Archives/edgar/data/1373715/000137371525000126/now-20250331.htm
Downloading: https://www.sec.gov/Archives/edgar/data/1373715/000137371524000344/now-20240930.htm

Saved 10-K files:
2025-01-30 10-K → sec_10k/10-K_000137371525000010_now-20241231.htm
2024-01-25 10-K → sec_10k/10-K_000137371524000030_now-20231231.htm

Saved 10-Q files:
2025-10-30 10-Q → sec_10q/10-Q_000137371525000309_now-20250930.htm
2025-07-24 10-Q → sec_10q/10-Q_000137371525000276_now-20250630.htm
2025-04-23 10-Q → sec_10q/10-Q_000137371525000126_now-20250331.htm
2024-10-24 10-Q → sec_10q/10-Q_00013737152400